In [ ]:
import sys
sys.path.append(r"./python")

from model import init_model_bedrock

model_name = "anthropic.claude-3-sonnet-20240229-v1:0"
llm = init_model_bedrock(model_name)

In [ ]:
from tqdm import tqdm
import fitz
from PIL import Image
import numpy as np
import base64

# prompt = """
# You are a document organizer of bicycle company and your task is to extract useful information from images.Please refer to the format of the content of the previous page to extract text information from the image on the current page.
# If the content on this page contains tables or maintenance process, please organize the tables or maintenance process into json format. 
# If the content of this page is a table and has no header, use the header of the previous page. 
# If the content of this page is a maintenance process and does not specify the specific maintenance object, use the maintenance object on the previous page.

# <previous page content>
# {context}
# </previous page content>
# No preface, just output the content directly.
# """

prompt = """
You are a document organizer of Research institute and your task is to extract useful information from images.
Please refer to the format of the content of the previous page to extract text information from the image on the current page.
If the content on this page contains tables or maintenance process, please organize the tables or maintenance process into json format. 
If the content of this page is a table and has no header, use the header of the previous page. 
If The text content at the front of the page has no title , use the title on the previous page.

<previous page content>
{context}
</previous page content>
No preface, just output the content directly.
"""

fname = '../docs/xxxx.pdf'
doc = fitz.open(fname)
previous_page_content = ''

for i in tqdm(range(doc.page_count)):
    page = doc.load_page(i)
    pix = page.get_pixmap()
    imgb64 = base64.b64encode(pix.tobytes()).decode("utf-8")
    #print('imgb64:',imgb64)
    model_kwargs = {'image': imgb64,'max_tokens':2048}
    llm.model_kwargs = model_kwargs
    new_prompt = prompt.format(context=previous_page_content)
    response = llm(prompt=new_prompt)
    previous_page_content = response
    print('response:',response)